<a href="https://colab.research.google.com/github/O-Kpy/Kaggle/blob/main/tabular_playground_series_jun_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import skew

In [ ]:
submit = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')
train = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv')
train.shape, test.shape, submit.shape

((200000, 77), (100000, 76), (100000, 10))

In [ ]:
train.target.value_counts().sort_index(ascending=False).plot.bar()

In [ ]:
data=pd.concat([train, test], axis=0)

In [ ]:
data.isnull().sum().sort_values(ascending=False)
# Null값 없고

In [ ]:
data.skew().sort_values(ascending=False)
# 스케일링 해야하고

In [ ]:
for i in train.drop(columns=['id', 'target']).columns:
    if train[i].skew() > 0.5:
        train[i] = np.log1p(train[i]+1)
        
for i in test.drop(columns=['id']).columns:
    if test[i].skew() > 0.5:
        test[i] = np.log1p(test[i]+1)

In [ ]:
train_agg = train.drop(columns='target').groupby('id').agg(['min', 'max', 'count', 'mean'])
train_agg_col = []

for i in train_agg.columns.levels[0]:
    for j in train_agg.columns.levels[1]:
        train_agg_col.append(f'{i}-{j}')
        
train_agg.columns = train_agg_col

test_agg = test.groupby('id').agg(['min', 'max', 'count', 'mean'])
test_agg_col = []

for i in test_agg.columns.levels[0]:
    for j in test_agg.columns.levels[1]:
        test_agg_col.append(f'{i}-{j}')
        
test_agg.columns = test_agg_col

In [ ]:
corr_matrix = train_agg.corr()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

to_drop = [c for c in upper.columns if any(abs(upper[c]) > 0.9)]

print(f'There are {len(train_agg.columns)} columns but {len(to_drop)} corr columns to remove')

There are 300 columns but 150 corr columns to remove


In [ ]:
train_agg.drop(columns=to_drop)
train = train.merge(train_agg, how='left', on='id')

test_agg.drop(columns=to_drop)
test = test.merge(test_agg, how='left', on='id')

# 그냥 모델링 하기 다음에 피쳐 엔지니어링

In [ ]:
from sklearn.metrics import log_loss, make_scorer
import catboost
import lightgbm
import xgboost
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, robust_scale, MinMaxScaler

In [ ]:
x_train = train.drop(columns=['target', 'id'])
y_train = train['target']
x_test = test.drop(columns=['id'])

In [ ]:
x_train = data.loc[data['target'].notnull()].drop(columns=['id', 'target'])
y_train = data.loc[data['target'].notnull()]['target']
x_test = data.loc[data['target'].isnull()].drop(columns=['id', 'target'])

In [ ]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_train = pd.Series(y_train)

In [ ]:
x_train_col = x_train.columns
x_test_col = x_test.columns

x_train = pd.DataFrame(MinMaxScaler().fit_transform(x_train), columns=x_train_col)
x_test = pd.DataFrame(MinMaxScaler().fit_transform(x_test), columns=x_test_col)

In [ ]:
# lgbm 모델
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
preds = np.zeros((len(x_test)))
for i, (train_index, valid_index) in enumerate(skf.split(x_train, y_train)):
  X_train, X_valid = x_train.iloc[train_index], x_train.iloc[valid_index]
  Y_train, Y_valid = y_train.iloc[train_index], y_train.iloc[valid_index]

  model = lightgbm.LGBMClassifier(n_estimators=3000, learning_rate=0.05, objective='multiclass', n_jobs=-1)
  model.fit(X_train, Y_train, eval_set=[(X_train, Y_train), (X_valid, Y_valid)], verbose=100, early_stopping_rounds=70, eval_metric='multi_logloss')
  preds = model.predict_proba(x_test)

Training until validation scores don't improve for 70 rounds
[100]	training's multi_logloss: 1.66235	valid_1's multi_logloss: 1.75672
[200]	training's multi_logloss: 1.59642	valid_1's multi_logloss: 1.75755
Early stopping, best iteration is:
[137]	training's multi_logloss: 1.63543	valid_1's multi_logloss: 1.75649
Training until validation scores don't improve for 70 rounds
[100]	training's multi_logloss: 1.66408	valid_1's multi_logloss: 1.74879
[200]	training's multi_logloss: 1.59782	valid_1's multi_logloss: 1.74931
Early stopping, best iteration is:
[130]	training's multi_logloss: 1.64222	valid_1's multi_logloss: 1.74837
Training until validation scores don't improve for 70 rounds
[100]	training's multi_logloss: 1.6637	valid_1's multi_logloss: 1.75045
Early stopping, best iteration is:
[114]	training's multi_logloss: 1.65283	valid_1's multi_logloss: 1.75018
Training until validation scores don't improve for 70 rounds
[100]	training's multi_logloss: 1.66313	valid_1's multi_logloss: 1.7

In [ ]:
# catboost모델 제일 성능이 잘 나옴(NN빼고)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
preds_cat = np.zeros((x_test.shape[0], y_train.nunique()))
for i, (train_index, valid_index) in enumerate(skf.split(x_train, y_train)):
    X_train, X_valid = x_train.iloc[train_index], x_train.iloc[valid_index]
    Y_train, Y_valid = y_train.iloc[train_index], y_train.iloc[valid_index]
    
    model = catboost.CatBoostClassifier(n_estimators=3000, learning_rate=0.03, objective='MultiClass', eval_metric='MultiClass', task_type='GPU', devices='0:1')
    model.fit(X_train, Y_train, verbose=100, early_stopping_rounds=70 )
    preds_cat += model.predict_proba(x_test)/skf.n_splits


In [ ]:
model_cat.select_features(
                X_train,
                Y_train,
                eval_set=[(X_train, Y_train), (X_valid, Y_valid)],
                features_for_select=None,
                num_features_to_select=None,
                algorithm='RecursiveByLossFunctionChange',
                steps=None,
                shap_calc_type=None,
                train_final_model=False,
                verbose=None,
                logging_level=None,
                plot=True)

In [ ]:
from catboost.utils import get_gpu_device_count
get_gpu_device_count()

In [ ]:
# xgboost ==> 너무 느림
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
preds_xgb = np.zeros((len(x_test)))
for i, (train_index, valid_index) in enumerate(skf.split(x_train, y_train)):
  X_train, X_valid = x_train.iloc[train_index], x_train.iloc[valid_index]
  Y_train, Y_valid = y_train.iloc[train_index], y_train.iloc[valid_index]

  model_xgb = xgboost.XGBClassifier(n_estimators=3000, learning_rate=0.022, objective='multiclass')
  model_xgb.fit(X_train, Y_train, eval_set=[(X_train, Y_train), (X_valid, Y_valid)], verbose=100, early_stopping_rounds=70, eval_metric='mlogloss')
  preds_xgb = model_xgb.predict_proba(x_test)

In [ ]:
submit = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv')
submit = submit.set_index('id')

In [ ]:
for i in range(9):
    submit.iloc[:,i] = preds_cat[:,i]

In [ ]:
submit.to_csv('submit8.csv')

# NN 파이토치 필사본

In [ ]:
import os
import sys
import datetime

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import optuna
from optuna.trial import TrialState

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=42)

for train_index, val_index in sss.split(x_train, y_train):
    print(f'train size:{len(train_index)}')
    print(f'val size:{len(val_index)}')
    X_train, X_val = x_train.iloc[train_index], x_train.iloc[val_index]
    Y_train, Y_val = y_train.iloc[train_index], y_train.iloc[val_index]

train size:140000
val size:60000
train size:140000
val size:60000
train size:140000
val size:60000
train size:140000
val size:60000
train size:140000
val size:60000


# Optuna

In [ ]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install skorch

     |████████████████████████████████| 128 kB 1.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import optuna
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
CLASSES = y_train.nunique()
NUM_FEATURES = x_train.shape[1]
DEVICE = torch.device("cpu" if not torch.cuda.is_available() else "cuda")

BATCHSIZE = 1000
EPOCHS = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 14
N_VALID_EXAMPLES = BATCHSIZE * 6

criterion = nn.CrossEntropyLoss()

def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 10)
    layers = []

    in_features = NUM_FEATURES
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), y_train.nunique(), 1024)
        layers.append(nn.Linear(in_features, out_features, bias=False))
        layers.append(nn.ReLU(inplace=True))
        p = trial.suggest_float("dropout_l{}".format(i), 0.1, 0.5)
        layers.append(nn.Dropout(p))
        layers.append(nn.BatchNorm1d(out_features))

        in_features = out_features
        
    layers.append(nn.Linear(in_features, CLASSES))

    return nn.Sequential(*layers)

def get_data(X_train, y_train, X_val, y_val):
    train = TensorDataset(torch.Tensor(np.array(X_train)), torch.Tensor(np.array(y_train)))
    train_loader = DataLoader(train, batch_size = 10000, shuffle = True)

    val = TensorDataset(torch.Tensor(np.array(X_val)), torch.Tensor(np.array(y_val)))
    val_loader = DataLoader(val, batch_size = 10000, shuffle = True)

    return train_loader, val_loader

def objective(trial):
    model = define_model(trial).to(DEVICE)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", 'AdamW', "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 0.001, 0.5)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)
    train_loader, valid_loader = get_data(X_train, Y_train, X_val, Y_val)
    
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            
            loss = criterion(output, target.long())
            loss.backward()
            optimizer.step()
            lr_scheduler.step(epoch + batch_idx / len(train_loader))

        # Validation of the model.
        model.eval()
        val_loss = []
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data)
                
                loss = criterion(output, target.long())
                val_loss.append(loss.item())
        
        avg_val_loss = np.mean(val_loss)

        trial.report(avg_val_loss, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    
    return avg_val_loss

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

[I 2021-06-25 09:32:43,891] A new study created in memory with name: no-name-973d2513-5ddf-4f25-ab71-29ada33c61b4
[I 2021-06-25 09:33:27,727] Trial 0 finished with value: 114.02726491292317 and parameters: {'n_layers': 6, 'n_units_l0': 130, 'dropout_l0': 0.1428306849730681, 'n_units_l1': 393, 'dropout_l1': 0.396970117149559, 'n_units_l2': 106, 'dropout_l2': 0.36191710858523607, 'n_units_l3': 690, 'dropout_l3': 0.1695344048295732, 'n_units_l4': 480, 'dropout_l4': 0.2774807448471288, 'n_units_l5': 898, 'dropout_l5': 0.3993355981749753, 'optimizer': 'RMSprop', 'lr': 0.16651074769676258}. Best is trial 0 with value: 114.02726491292317.
[I 2021-06-25 09:34:11,192] Trial 1 finished with value: 1.8835017482439678 and parameters: {'n_layers': 4, 'n_units_l0': 237, 'dropout_l0': 0.12150456098751161, 'n_units_l1': 257, 'dropout_l1': 0.11908673810631445, 'n_units_l2': 852, 'dropout_l2': 0.4415032210004828, 'n_units_l3': 924, 'dropout_l3': 0.42800490585728224, 'optimizer': 'SGD', 'lr': 0.185452025

KeyboardInterrupt: 

In [ ]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print('Study statistics:')
print(' Number of finished trials:', len(study.trials))
print(' Number of pruned trials:', len(pruned_trials))
print(' Number of complete trials:', len(complete_trials))

print('Best trial:')
trial = study.best_trial

print(' Value:', trial.value)
print(' Params:')
for key, value in trial.params.items():
    print(f'{key}:{value}')

Study statistics:
 Number of finished trials: 50
 Number of pruned trials: 19
 Number of complete trials: 31
Best trial:
 Value: 1.840059796969096
 Params:
n_layers:4
n_units_l0:510
dropout_l0:0.36217530281546845
n_units_l1:636
dropout_l1:0.4127331955172506
n_units_l2:263
dropout_l2:0.497500273667107
n_units_l3:1010
dropout_l3:0.21133027314680752
optimizer:RMSprop
lr:0.0009986857150353628


In [ ]:
optuna.visualization.plot_intermediate_values(study).show()

# 최적화된 파라미터를 가지고 모델링

In [ ]:
class classification_model(nn.Module):
    def __init__(self, n_in, n_out, layers, p=None):
        super(classification_model, self).__init__()
        
        all_layers=[]
        self.n_in = n_in
        self.n_out = n_out
        
        for i in range(len(layers)):
            all_layers.append(nn.Linear(self.n_in, layers[i], bias=False))
            all_layers.append(nn.ReLU(inplace=True))
            if p:
                all_layers.append(nn.Dropout(p[i]))
            all_layers.append(nn.BatchNorm1d(layers[i]))
            
            self.n_in = layers[i]
            
        all_layers.append(nn.Linear(layers[-1], self.n_out))
        
        self.layers=nn.Sequential(*all_layers)
        
    def forward(self, x):
        x = self.layers(x)
        return x
    
    net_name='classification_model'

In [ ]:
batch_size = 1000

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=0)

for train_index, val_index in sss.split(x_train, y_train):    
    print("TRAIN:\t", train_index, "Size:\t", len(train_index))
    print("VAL:\t", val_index, "Size:\t", len(val_index))
    X_train, X_val = x_train.iloc[train_index], x_train.iloc[val_index]
    Y_train, Y_val = y_train.iloc[train_index], y_train.iloc[val_index]

train = TensorDataset(torch.Tensor(np.array(X_train)), torch.Tensor(np.array(Y_train)))
train_loader = DataLoader(train, batch_size = batch_size, shuffle = True)

val = TensorDataset(torch.Tensor(np.array(X_val)), torch.Tensor(np.array(Y_val)))
val_loader = DataLoader(val, batch_size = batch_size, shuffle = True)

phases = ["train", "val"]
loaders = {"train": train_loader, "val": val_loader}

TRAIN:	 [ 80553  41670 142223 ... 154852 130289  53137] Size:	 140000
VAL:	 [ 75883  65709  45309 ... 181524  14476   2577] Size:	 60000


In [ ]:
"""
Best trial:
 Value: 1.7687533100446065
 Params:
n_layers:1
n_units_l0:880
dropout_l0:0.2116281048320166
optimizer:RMSprop
lr:0.0009784821223970022
"""

layers_=[]
dropout_=[]
l_rate=trial.params['lr']

for i in range(trial.params['n_layers']):
    layers_.append(trial.params[f'n_units_l{i}'])
    dropout_.append(trial.params[f'dropout_l{i}'])
    
device = torch.device('cpu' if not torch.cuda.is_available() else 'cuda')

model = classification_model(x_train.shape[1], y_train.nunique(), layers_, p=dropout_)
model.to(device)

criterion = nn.CrossEntropyLoss()
if trial.params['optimizer'] == 'RMSprop':
    optimizer = optim.RMSprop(model.parameters(), lr=l_rate)
elif trial.params['optimizer'] == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=l_rate)
elif trial.params['optimizer'] == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=l_rate)
    
print(device)
print(model)

cuda
classification_model(
  (layers): Sequential(
    (0): Linear(in_features=375, out_features=510, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.36217530281546845, inplace=False)
    (3): BatchNorm1d(510, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=510, out_features=636, bias=True)
    (5): ReLU(inplace=True)
    (6): Dropout(p=0.4127331955172506, inplace=False)
    (7): BatchNorm1d(636, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Linear(in_features=636, out_features=263, bias=True)
    (9): ReLU(inplace=True)
    (10): Dropout(p=0.497500273667107, inplace=False)
    (11): BatchNorm1d(263, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Linear(in_features=263, out_features=1010, bias=True)
    (13): ReLU(inplace=True)
    (14): Dropout(p=0.21133027314680752, inplace=False)
    (15): BatchNorm1d(1010, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    

In [ ]:
loss_train=[]
loss_valid=[]
best_validation_loss = 1000
best_epoch=1
correct=0
total=0

n_epochs=50

now = datetime.datetime.now()
weights_path = "./output/{:%Y%m%dT%H%M}".format(now)
os.makedirs(weights_path, exist_ok=True)

lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)
break_st = False

for epoch in range(1, n_epochs+1):
    
    for phase in phases:
        if phase == 'train':
            model.train()
        elif phase == 'val':
            model.eval()
        
        for _, data in enumerate(loaders[phase], 0):
            features, y_true = data[0], data[1]
            features = features.to(device, dtype=torch.float)
            y_true = y_true.to(device, dtype=torch.float)
            
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(phase=='train'):
                y_pred = model(features)
                
                sm = nn.Softmax(dim=1)  # predict_proba
                pred_percentage = sm(y_pred)
                
                if break_st:
                    break
                    
                y_true = y_true.long()
                
                _, preds = torch.max(pred_percentage, 1)
                total += y_true.size(0)
                correct += (preds == y_true).sum().item()
                
                loss = criterion(y_pred, y_true)
                
                if phase == 'val':
                    loss_valid.append(loss.item())
                    
                if phase == 'train':
                    loss_train.append(loss.item())
                    loss.backward()
                    
                    optimizer.step()
                    lr_scheduler.step(epoch / len(train_loader))
        if break_st:
            break
        if phase == 'train':
            mean_train_loss = np.mean(loss_train)
            acc_train = 100*correct/total
            loss_train = []
            correct = 0
        
        if phase == 'val':
            validation_loss = np.mean(loss_valid)
            acc_valid = 100*correct/total
            loss_valid = []
            correct = 0
            total = 0
                
    if break_st:
        break
    if validation_loss < best_validation_loss:
        print('saving weights...')
        best_epoch = epoch
        best_validation_loss = validation_loss
        torch.save(model.state_dict(),
                   os.path.join(weights_path, 'model.pt'),)
            
    print(f'Epoch={epoch}/{n_epochs}\tloss={mean_train_loss:.4f}\tval_loss={validation_loss:.4f}\tacc={acc_train:.4f}\tval_acc={acc_valid:.4f}')
                


saving weights...
Epoch=1/50	loss=1.7158	val_loss=1.7763	acc=37.7643	val_acc=10.6440
Epoch=2/50	loss=1.7135	val_loss=1.7776	acc=37.9057	val_acc=10.6425
saving weights...
Epoch=3/50	loss=1.7119	val_loss=1.7763	acc=37.9364	val_acc=10.6585
Epoch=4/50	loss=1.7134	val_loss=1.7777	acc=37.9450	val_acc=10.6515
Epoch=5/50	loss=1.7122	val_loss=1.7763	acc=37.8850	val_acc=10.6635
Epoch=6/50	loss=1.7130	val_loss=1.7770	acc=37.9743	val_acc=10.6545
Epoch=7/50	loss=1.7139	val_loss=1.7788	acc=37.8364	val_acc=10.6545
Epoch=8/50	loss=1.7128	val_loss=1.7785	acc=37.9429	val_acc=10.6610
saving weights...
Epoch=9/50	loss=1.7124	val_loss=1.7763	acc=37.9436	val_acc=10.6505
Epoch=10/50	loss=1.7137	val_loss=1.7769	acc=37.9157	val_acc=10.6510
saving weights...
Epoch=11/50	loss=1.7134	val_loss=1.7756	acc=37.8807	val_acc=10.6705
Epoch=12/50	loss=1.7123	val_loss=1.7768	acc=38.0221	val_acc=10.6510
saving weights...
Epoch=13/50	loss=1.7126	val_loss=1.7750	acc=37.9921	val_acc=10.6590
Epoch=14/50	loss=1.7128	val_loss=1.

In [ ]:
test = TensorDataset(torch.Tensor(np.array(x_test)))
test_loader = DataLoader(test, batch_size=100000, shuffle=False)

In [ ]:
if not weights_path:
    print('Choose weights path')
    sys.exit()

device = torch.device('cpu' if not torch.cuda.is_available() else 'cuda')

model_name = weights_path + '/model.pt'
print(model_name)
state_dict = torch.load(model_name, map_location=device)
model.load_state_dict(state_dict)
model.eval()

for _, data in enumerate(test_loader, 0):
    features = data[0]
    print(features.size())
    features = features.to(device, dtype=torch.float)
    
    with torch.set_grad_enabled(False):
        y_pred = model(features)
        
        sm = nn.Softmax(dim=1)
        pred_percentage = sm(y_pred)
        
        print(pred_percentage.size())

print(pred_percentage.detach().cpu().numpy())
print('Done!')

./output/20210625T0915/model.pt
torch.Size([100000, 375])
torch.Size([100000, 9])
[[0.08042812 0.472009   0.17498638 ... 0.01489316 0.03152972 0.10974982]
 [0.02847469 0.01827991 0.01802572 ... 0.08047377 0.40378097 0.08335941]
 [0.01908431 0.0219461  0.01843582 ... 0.03277696 0.14273822 0.05185917]
 ...
 [0.06526197 0.3491881  0.15484692 ... 0.03494824 0.08958131 0.13881981]
 [0.02777312 0.01361246 0.01413754 ... 0.0782783  0.3770459  0.0729562 ]
 [0.03722076 0.04652073 0.05035622 ... 0.091174   0.29906538 0.12914579]]
Done!


In [ ]:
submit = pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')
pred_array = pred_percentage.detach().cpu().numpy()
submit.loc[:, 'Class_1':'Class_9'] = pred_array
submit = submit.set_index('id')
submit.head()

,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
id,,,,,,,,,
200000,0.080428,0.472009,0.174986,0.024173,0.012897,0.079334,0.014893,0.031530,0.109750
200001,0.028475,0.018280,0.018026,0.008878,0.007393,0.351335,0.080474,0.403781,0.083359
200002,0.019084,0.021946,0.018436,0.009017,0.008079,0.696063,0.032777,0.142738,0.051859
200003,0.041159,0.075034,0.069922,0.028387,0.019746,0.275462,0.083476,0.247847,0.158967
200004,0.037041,0.061479,0.060587,0.024905,0.018075,0.293186,0.084864,0.271702,0.148162


In [ ]:
submit.to_csv('./output/submission10.csv')

# NN모델을 써야 점수가 올라간다. 
# Pytorch NN모델을 생성하고 제출하자
# feature_select할게 더 있나? 체크